# Hovmoller Temperature Depth

This notebook shows how to calculate a depth-time Hovmoller plot of globally averaged potential temperature from ACCESS-OM2.

First, the preliminaries, including starting a cluster of 8 workers to do the calculation.

In [1]:
%matplotlib inline

import pandas as pd
import intake
catalog = intake.cat.access_nri
import matplotlib.pyplot as plt
import numpy as np
import cmocean as cm
import IPython.display
from dask.distributed import Client



**Select an Experiment:**

In [2]:
client = Client("tcp://10.6.22.43:8786") 
client

<Client: 'tcp://10.6.22.43:8786' processes=1 threads=48, memory=188.56 GiB>

In [3]:
expt = '1deg_jra55_ryf9091_gadi'

Now, we load an entire temperature dataset, along with the corresponding WOA13 temperature dataset to compute the temperature anomaly.

In [4]:
cat_subset = catalog[expt]
var_search = cat_subset.search(variable='temp',frequency='1mon')
darray = var_search.to_dask()
darray = darray['temp']
temp = darray
cat_subset = catalog['woa13/10']
var_search = cat_subset.search(variable='temp',frequency='1mon')
darray = var_search.to_dask()
darray = darray['temp']
temp_WOA13 = darray.mean('time').compute()
#temp_WOA13 = cc.get_nc_variable('woa13/025', 'woa13_ts_??_mom025.nc', 'temp').mean('time').compute()
temp_anom = temp - temp_WOA13.values - 273.15
IPython.display.clear_output()

/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)
/g/data/hh5/public/apps/miniconda3/envs/analysis3-23.07/lib/python3.10/site-packages/intake_esm/cat.py:270: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  .applymap(type)


KeyError: "key='woa13/10' not found in catalog. You can access the list of valid source keys via the .keys() method."

Next, we load cell area (denoted $a(x,y,z)$) from the ocean_grid file, construct a mask from the temperature file and make a profile of the total ocean area as a function of depth, $A$, namely
$$ A(z) = \sum_x \sum_y a(x,y,z)$$
*Note that this mask ignores the presence of partial cells, for now*.

In [ ]:
cat_subset = catalog[expt]
var_search = cat_subset.search(variable='area_t')
var_search = var_search.search(path=var_search.df['path'][0])
darray = var_search.to_dask()
darray = darray['area_t']
area_t = darray.mean('time')
mask = temp.isel(time=0).copy()
mask = mask/mask                 ## This seems pretty dodgy to me, but it works!!
area = mask*area_t
area_sum = area.sum('xt_ocean').sum('yt_ocean')
IPython.display.clear_output()

Now, the mean temperature at each time level can then be computed as 
$$T(z,t) = \frac{\sum_x \sum_y a(x,y,z) \theta(x,y,z,t)}{A(z)}$$
where $T$ is the average temperature and $\theta$ the potential temperature.

In [ ]:
var =area_t*temp_anom
temp_hov = var.sum('xt_ocean').sum('yt_ocean').compute()
temp_hov = temp_hov/area_sum

We can use xarray for a quick and dirty plot of this data:

In [ ]:
temp_hov.T.plot.contourf(levels=25,yincrease=False)

Alternatively, we could put a little more effort into plotting, including expansion of the upper 1000m:

In [ ]:
t_up = temp_hov.sel(st_ocean=slice(0,1000))
levs = np.arange(-1,1,0.1)
fig,ax = plt.subplots(2,1,sharex=True,figsize=(6,6))
plt.subplots_adjust(hspace=0,right=0.85)

t_up.T.plot.contourf(ax=ax[0],levels=levs,cmap = cm.cm.balance,yincrease=False,add_colorbar=False)
ax[0].set_ylabel('')

t_lo = temp_hov.sel(st_ocean=slice(1000,6000))
p1=t_lo.T.plot.contourf(ax=ax[1],levels=levs,cmap = cm.cm.balance,yincrease=False,add_colorbar=False)
plt.ylabel('Depth (m)')
plt.xlabel('Time (yrs)')

ax1 = plt.axes([0.88,0.3,0.02,0.4])
cb = plt.colorbar(p1,cax=ax1,orientation='vertical')
ax1.xaxis.set_label_position("top")
cb.ax.set_xlabel('°C')

ax[0].set_title('Horizontally averaged evolution of potential temperature')
